## Day 19

In [1]:
def readInput19(filename):
    with open(filename) as f:
        p = [ [ c for c in l.strip("\n").split("\n") ]  for l in f.read().split("\n\n") ]
        scanners = []
        for s in p:
            nscanner = s[0].replace("--- scanner ","").replace(" ---","")
            coords = []
            for c in s[1:]:
                coord = tuple([int(x) for x in c.split(",")])
                coords.append(coord)
            scanners.append(coords)
        return scanners

### Rotation matrices 

Generating rotation matrices using Euler angles...

In [168]:
import numpy as np
from math import cos,sin,pi
  
def Rx(theta):
    return np.matrix([[ 1, 0         , 0         ],
                      [ 0, cos(theta),-sin(theta)],
                      [ 0, sin(theta), cos(theta)]])
  
def Ry(theta):
    return np.matrix([[ cos(theta), 0, sin(theta)],
                      [ 0         , 1, 0         ],
                      [-sin(theta), 0, cos(theta)]])
  
def Rz(theta):
    return np.matrix([[ cos(theta), -sin(theta), 0 ],
                      [ sin(theta),  cos(theta), 0 ],
                      [ 0         , 0          , 1 ]])

def R(psi,theta,phi):
     return Rz(psi) * Ry(theta) * Rx(phi)

def getRotations():
    Rotations = []
    for i in range(4):
        for j in range(4):
            for k in range(4):
                phi   = i*np.pi/2
                theta = j*np.pi/2
                psi   = k*np.pi/2
                Rot = np.array(np.round(R(psi,theta,phi),decimals=2),dtype=int)
                unique = True
                for Rot0 in Rotations:
                    if (Rot==Rot0).all():
                        unique = False
                if unique:
                    Rotations.append(Rot)
    return Rotations

Rotations = getRotations()
print("Generated {} rotation matrices".format(len(Rotations)))

Generated 24 rotation matrices


### Testing rotation matrices on first example...

In [174]:
rotTest = readInput19("data/day19test1.txt")

for R in Rotations:
    foundRotation = 0
    for v0,v1 in zip(rotTest[0],rotTest[2]):
        X0 = np.array(v0)
        X1 = np.array(v1)
        X = np.matmul(R,X0)
        if (X==X1).all():
            foundRotation += 1
    if foundRotation==len(rotTest[0]):
        print(R)

[[ 0  0 -1]
 [ 0  1  0]
 [ 1  0  0]]


### Pair-wise distances

In principle I don't think I need to rotate the coordinates from a given scanner to match those from another scanner, I could compute the distances between pairs of beacons in their reference frame, and these distances would not depend on the frame orientation. Then I could possibly match the pair-wise distances from different scanners (under the assumption the input is not too evil!).

In order to avoid rounding problem, I can save the distance square instead of taking the square root.

In the example each scanner measures 25 beacons, so I'll have 25*24/2 = 300 pair-wise distances.

Each pair of scanner sees at least 12 common beacons, that would correspond to the 12*11/2 = 66 common pair-wise distances. I can check the overlap using `set.intersection`. 

In [175]:
from itertools import combinations
from collections import defaultdict

def squareDistance(a,b):
    return sum([ (xa-xb)**2 for xa,xb in zip(a,b) ])

def distances(scanner):
    return [ [ squareDistance(a,b) for a,b in combinations(s,2) ] for s in scanner ]

def findOverlappingScans(scanner):
    overlapping = defaultdict(list)
    dists = distances(scanner)
    for i,j in combinations(range(len(dists)),2):
        d1 = dists[i]
        d2 = dists[j]
        common = list(set(d1).intersection(set(d2)))
        if len(common)>=66:
            overlapping[i].append(j)
            overlapping[j].append(i)
    return overlapping

In [176]:
scanner0 = readInput19("data/day19test2.txt")
overlapping = findOverlappingScans(scanner0)
overlapping

defaultdict(list, {0: [1], 1: [0, 3, 4], 3: [1], 4: [1, 2], 2: [4]})

### Finding commong beacons and corresponding rotation and translation

In [228]:
def findCommonBeacons(beacons1,beacons2,verbose=True):
    c1 = [ (a,b) for a,b in combinations(beacons1,2) ]
    c2 = [ (a,b) for a,b in combinations(beacons2,2) ]
    d1 = [ squareDistance(a,b) for a,b in combinations(beacons1,2) ]
    d2 = [ squareDistance(a,b) for a,b in combinations(beacons2,2) ]
    common = list(set(d1).intersection(set(d2)))
    if len(common) >= 66:
        if verbose: print("Found overlapping region ",end="")
        C1 = []
        C2 = []
        for c in common:
            i1 = d1.index(c)
            i2 = d2.index(c)
            for x1 in c1[i1]:
                if x1 not in C1:
                    C1.append(x1)
            for x2 in c2[i2]:
                if x2 not in C2:
                    C2.append(x2)
        if verbose: print("with {} common beacons.".format(len(C1)))
        if verbose: print("Looking for rotation and scanner distance... ",end="")
        # Taking one point as reference
        X1 = np.array(C1[0])
        R  = None
        dX = None
        # Trying all points in corresponding set to find match
        matchFound = False
        for x2 in C2:
            X2 = np.array(x2)
            # Trying all rotations
            for R in Rotations:
                X2rot = np.matmul(R,X2)
                # Translation between reference point X1 and test point X2 after rotation
                dX = X1-X2rot
                # If point and rotation correct, than all other points should have a correspondence in original set
                C2transf = [ tuple(np.matmul(R,np.array(x2t))+dX) for x2t in C2 ]
                inOriginal = [ 1 for x2t in C2transf if x2t in C1 ]
                # There might be more than 12 elements in C1 for spurious matching of
                # distances, only check for lenght of inOriginal to be at least 12
                #if len(inOriginal)==len(C1):
                if len(inOriginal)>=12:
                    if verbose: print("found!")
                    if verbose: print(R,dX)
                    matchFound = True
                    break
            if matchFound: break
        if not matchFound:
            return [], (0,0,0)
        if verbose: print("Translating all points in second set to coordinate of first set... ",end="")
        beacon2new = [ tuple(np.matmul(R,np.array(x2))+dX) for x2 in beacons2 ]
        if verbose: print("Done.")
        return beacon2new, tuple(dX)
    else:
        if verbose: print("No overlapping region found.")
        return [], (0,0,0)

In [230]:
scanner0 = readInput19("data/day19test2.txt")

b1 = scanner0[0]
b2 = scanner0[1]
b2new, Xscanner2 = findCommonBeacons(b1,b2,verbose=True)

print(len(b1),len(b2),len(set(b1)|set(b2new)))
print(Xscanner2)

Found overlapping region with 12 common beacons.
Looking for rotation and scanner distance... found!
[[-1  0  0]
 [ 0  1  0]
 [ 0  0 -1]] [   68 -1246   -43]
Translating all points in second set to coordinate of first set... Done.
25 25 38
(68, -1246, -43)


### Part 1: solving overlaps between all regions, translating to first scanner coordinates, remove overlaps

In [237]:
def solveOverlaps(scanner):
    scanner_new = [[]]*len(scanner)
    scanner_new[0] = list(scanner[0])
    scanner_coord = [(0,0,0)]*len(scanner)
    translated = [0]
    refs = [0]
    while len(translated)<len(scanner):
        iref = refs.pop()
        b1 = scanner_new[iref]
        for i in range(len(scanner)):
            if i!=iref and i not in translated:
                b2 = scanner[i]
                b2new, Xscanner = findCommonBeacons(b1,b2,verbose=False)
                if len(b2new):
                    scanner_new[i] = b2new
                    scanner_coord[i] = Xscanner
                    translated.append(i)
                    refs.append(i)
    return scanner_new, scanner_coord

In [245]:
scanner0 = readInput19("data/day19test2.txt")

scanner_new0, scanner_coord0 = solveOverlaps(scanner0)

def part1(scanner_new):
    overlap = set([])
    for b in scanner_new:
        overlap = overlap | set(b)
    return len(overlap)

print("Test 1:",part1(scanner_new0))

Test 1: 79


In [246]:
scanner = readInput19("data/input19.txt")

scanner_new, scanner_coord = solveOverlaps(scanner)

print("Part 1:",part1(scanner_new))

Part 1: 434


### Part 2: computing Manhattan distances between all scanners, find largest

Modified `findCommonBeacons` function to also return scanner coordinates

In [240]:
def manhattanDistance(a,b):
    return sum([abs(xa-xb) for xa,xb in zip(a,b)])

# In total, they are 1197 + 1175 + 1249 = 3621 units apart.

a = (1105,-1205,1229)
b = (-92,-2380,-20)

manhattanDistance(a,b)

3621

In [243]:
def part2(scanner_coord):
    return max([ manhattanDistance(a,b) for a,b in combinations(scanner_coord,2) ] )  

In [249]:
print("Test 2:",part2(scanner_coord0))

Test 2: 3621


In [250]:
print("Part 2:",part2(scanner_coord))

Part 2: 11906
